# Comprehensive Implementation of Graph RAG for Job Profile Analysis Using Local Ollama Models

This notebook demonstrates a complete Graph RAG implementation for analyzing job profile documents using local Ollama models.

## 1. Environment Setup

In [1]:
# !pip install -qU langchain langchain_community langchain-experimental neo4j pyvis ollama python-dotenv
# !ollama pull llama3.1  # 8B parameter model recommended
# !ollama pull nomic-embed-text  # Embedding model

import os
import sys
sys.path.append(os.path.abspath('..'))

## 2. Document Processing Pipeline

In [2]:

TESTING=False           # Set to True to run with toy data
RUN_IN_BATCH=True       # If False, will try to run all chunks at once to get entity connections, otherwise will save after each one
USE_OLLAMA=False         # Whether to use local Ollama or Azure API
LIMIT_CHUNKS=1       # Set to a number to limit the number of chunk to be processed
CHUNK_SIZE=1000          
csvPath="../data/job profiles/2025-02-07_profiles.csv"

Load data:

In [4]:
from typing import List, Optional
from pydantic import BaseModel, Field
import pandas as pd
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.messages import UserMessage

class JobProfile(BaseModel):
    title: str = Field(description="Official job title")
    classifications: List[str] = Field(description="Classification codes")
    organizations: List[str] 
    behavioural_competencies: List[str]
    education: List[str] = Field(description="Education requirements")
    job_experience: List[str]
    knowledge_skills_abilities: List[str]
    security_screenings: List[str]
    accountabilities: List[str]
    role_type: Optional[str] = Field(description="Role category")
    scopes: Optional[List[str]] = Field(description="Areas of responsibility")
    professional_registration: Optional[List[str]]

if not TESTING:
    from notebooks.utils import get_job_profile_documents
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from transformers import AutoTokenizer

    documents=get_job_profile_documents(csvPath, include_org_class_sections=False)
    tokenizer = MistralTokenizer.from_model("mistral-small", strict=True)

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=200,
        separators=["\n\n", "\n", "•", " ", ""],
        length_function=lambda text: len(tokenizer.encode_chat_completion(
            ChatCompletionRequest(
                messages=[
                    UserMessage(content=text)
                ],
                model="mistral-small-latest"
            )
        ).tokens)
    )
    
    chunks = splitter.split_documents(documents)


In [5]:
if LIMIT_CHUNKS is not None:
    chunks=chunks[0:LIMIT_CHUNKS]

In [6]:
csvPath="../data/job profiles/2025-02-07_profiles.csv"
df=pd.read_csv(csvPath)
df.columns

Index(['id', 'version', 'title', 'number', 'overview', 'program_overview',
       'state', 'type', 'behavioural_competencies', 'accountabilities',
       'education', 'job_experience', 'professional_registration_requirements',
       'preferences', 'knowledge_skills_abilities', 'willingness_statements',
       'optional_requirements', 'security_screenings', 'all_reports_to',
       'context', 'is_archived', 'valid_from', 'valid_to', 'views', 'role',
       'role_type', 'created_at', 'updated_at', 'published_at',
       'classifications', 'organizations', 'scopes', 'job_families', 'streams',
       'reports_to'],
      dtype='object')

In [7]:
from langchain.globals import set_debug
# import logging

# Enable verbose logging for all components
set_debug(True)

# import logging
# logging.basicConfig(level=logging.INFO) # DEBUG

## 4. Neo4j Graph Database Integration

Initialize db connection

In [9]:
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
import os

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "your_password"

graph = Neo4jGraph()
print(graph.query("CALL db.info()"))

[{'id': '69116869CB35145331EE2540ACCB41CE6769FA0EF563941B7EF46E8944A02587', 'name': 'neo4j', 'creationDate': '2025-02-23T22:56:33.286Z'}]


### Load structured data into neo4j

In [10]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from notebooks.utils import get_clean_job_profiles_df


df = get_clean_job_profiles_df()

In [11]:
print(df.iloc[0])

id                                                                                       13
version                                                                                   1
title                                                                       Licensing Clerk
number                                                                                  192
overview                                  To process all new license applications or ren...
program_overview                                                                        NaN
state                                                                             PUBLISHED
type                                                                              CORPORATE
behavioural_competencies                  [Decisive insight, Information seeking, Concer...
accountabilities                          [Reviews licensing applications to ensure they...
education                                 [Certificate or coursework and 6 month

In [12]:
from neo4j import GraphDatabase
import pandas as pd
import json
from tqdm import tqdm

def load_job_profiles_to_neo4j(df, uri, username, password):
    """
    Load job profiles data from DataFrame to Neo4j
    
    Parameters:
    df: pandas DataFrame containing job profiles
    uri: Neo4j connection URI (e.g., "neo4j://localhost:7687")
    username: Neo4j username
    password: Neo4j password
    """
    # Connect to Neo4j
    driver = GraphDatabase.driver(uri, auth=(username, password))
    
    with driver.session() as session:
        print("Deleting all existing data...")
        session.run("MATCH (n) DETACH DELETE n")
        
        # Create constraints and indexes for better performance
        session.run("CREATE CONSTRAINT job_profile_id IF NOT EXISTS FOR (j:JobProfile) REQUIRE j.id IS UNIQUE")
        session.run("CREATE INDEX job_profile_title IF NOT EXISTS FOR (j:JobProfile) ON (j.title)")
        
        # Create indexes for related nodes
        for label in ["BehavioralCompetency", "Accountability", "Education", "Experience", 
                     "Registration", "Preference", "KSA", "WillingnessStatement", 
                     "OptionalRequirement", "SecurityScreening", "Role", "RoleType", 
                     "Classification", "Organization", "Scope", "JobFamily", "Stream", "ReportsTo"]:
            session.run(f"CREATE INDEX {label.lower()}_name IF NOT EXISTS FOR (n:{label}) ON (n.name)")
        
        # Process each job profile
        # i=0
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Loading job profiles"):
            # i+=1
            # if i>10:
            #     break
            # print('processing: ', i, '/', len(df))
            
            # Create job profile node
            create_job_profile_query = """
            MERGE (j:JobProfile {id: $id})
            SET j.version = $version,
                j.title = $title,
                j.number = $number,
                j.overview = $overview,
                j.program_overview = $program_overview,
                j.state = $state,
                j.type = $type,
                j.context = $context,
                j.is_archived = $is_archived,
                j.all_reports_to = $all_reports_to,
                j.valid_from = datetime($valid_from),
                j.valid_to = CASE WHEN $valid_to IS NULL THEN NULL ELSE datetime($valid_to) END,
                j.views = $views,
                j.created_at = datetime($created_at),
                j.updated_at = datetime($updated_at),
                j.published_at = CASE WHEN $published_at IS NULL THEN NULL ELSE datetime($published_at) END
            RETURN j
            """
            
            # Convert timestamps to ISO format for Neo4j
            valid_from = row['valid_from'].isoformat() if pd.notna(row.get('valid_from')) else None
            valid_to = row['valid_to'].isoformat() if pd.notna(row.get('valid_to')) else None
            created_at = row['created_at'].isoformat() if pd.notna(row.get('created_at')) else None
            updated_at = row['updated_at'].isoformat() if pd.notna(row.get('updated_at')) else None
            published_at = row['published_at'].isoformat() if pd.notna(row.get('published_at')) else None
            
            # Create job profile node
            job_profile = session.run(
                create_job_profile_query,
                id=int(row['id']),
                version=int(row['version']) if pd.notna(row.get('version')) else None,
                title=row['title'] if pd.notna(row.get('title')) else None,
                number=int(row['number']) if pd.notna(row.get('number')) else None,
                overview=row['overview'] if pd.notna(row.get('overview')) else None,
                program_overview=row['program_overview'] if pd.notna(row.get('program_overview')) else None,
                state=row['state'] if pd.notna(row.get('state')) else None,
                type=row['type'] if pd.notna(row.get('type')) else None,
                context=row['context'] if pd.notna(row.get('context')) else None,
                is_archived=bool(row['is_archived']) if pd.notna(row.get('is_archived')) else False,
                all_reports_to=bool(row['all_reports_to']) if pd.notna(row.get('all_reports_to')) else False,
                valid_from=valid_from,
                valid_to=valid_to,
                views=int(row['views']) if pd.notna(row.get('views')) else 0,
                created_at=created_at,
                updated_at=updated_at,
                published_at=published_at
            ).single()
            
            # Create relationships for list fields
            create_relationships(session, row['id'], 'behavioural_competencies', 'BehavioralCompetency', 'HAS_COMPETENCY', row)
            create_relationships(session, row['id'], 'accountabilities', 'Accountability', 'HAS_ACCOUNTABILITY', row)
            create_relationships(session, row['id'], 'education', 'Education', 'REQUIRES_EDUCATION', row)
            create_relationships(session, row['id'], 'job_experience', 'Experience', 'REQUIRES_EXPERIENCE', row)
            create_relationships(session, row['id'], 'professional_registration_requirements', 'Registration', 'REQUIRES_REGISTRATION', row)
            create_relationships(session, row['id'], 'preferences', 'Preference', 'HAS_PREFERENCE', row)
            create_relationships(session, row['id'], 'knowledge_skills_abilities', 'KSA', 'REQUIRES_KSA', row)
            create_relationships(session, row['id'], 'willingness_statements', 'WillingnessStatement', 'HAS_WILLINGNESS', row)
            create_relationships(session, row['id'], 'optional_requirements', 'OptionalRequirement', 'HAS_OPTIONAL_REQUIREMENT', row)
            create_relationships(session, row['id'], 'security_screenings', 'SecurityScreening', 'REQUIRES_SCREENING', row)
            create_relationships(session, row['id'], 'role', 'Role', 'HAS_ROLE', row)
            create_relationships(session, row['id'], 'role_type', 'RoleType', 'HAS_ROLE_TYPE', row)
            create_relationships(session, row['id'], 'classifications', 'Classification', 'HAS_CLASSIFICATION', row)
            create_relationships(session, row['id'], 'organizations', 'Organization', 'BELONGS_TO_ORGANIZATION', row)
            create_relationships(session, row['id'], 'scopes', 'Scope', 'HAS_SCOPE', row)
            create_relationships(session, row['id'], 'job_families', 'JobFamily', 'BELONGS_TO_JOB_FAMILY', row)
            create_relationships(session, row['id'], 'streams', 'Stream', 'BELONGS_TO_STREAM', row)
            create_relationships(session, row['id'], 'reports_to', 'ReportsTo', 'REPORTS_TO', row)
    
    driver.close()
    print("Data loading completed successfully!")

def create_relationships(session, job_id, field_name, node_label, relationship_type, row):
    """Create relationships between job profile and related entities"""
    if isinstance(row.get(field_name), list) and len(row.get(field_name)) > 0:
        for item in row[field_name]:
            query = f"""
            MATCH (j:JobProfile {{id: $job_id}})
            MERGE (n:{node_label} {{name: $name}})
            MERGE (j)-[r:{relationship_type}]->(n)
            RETURN j, n
            """
            session.run(query, job_id=int(job_id), name=item)

In [13]:
URI = "neo4j://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "your_password"

# Load data to Neo4j
load_job_profiles_to_neo4j(df, URI, USERNAME, PASSWORD)

Deleting all existing data...


Loading job profiles: 100%|██████████| 502/502 [02:19<00:00,  3.61it/s]

Data loading completed successfully!


In [14]:
graph.query("CALL db.schema.visualization()")

[{'nodes': [{'name': 'Organization', 'indexes': ['name'], 'constraints': []},
   {'name': 'RoleType', 'indexes': ['name'], 'constraints': []},
   {'name': 'JobFamily', 'indexes': ['name'], 'constraints': []},
   {'name': 'Document', 'indexes': ['embedding'], 'constraints': []},
   {'name': 'JobProfile',
    'indexes': ['title'],
    'constraints': ["Constraint( id=7, name='job_profile_id', type='UNIQUENESS', schema=(:JobProfile {id}), ownedIndex=6 )"]},
   {'name': 'OptionalRequirement', 'indexes': ['name'], 'constraints': []},
   {'name': 'Role', 'indexes': ['name'], 'constraints': []},
   {'name': 'Experience', 'indexes': ['name'], 'constraints': []},
   {'name': 'Scope', 'indexes': ['name'], 'constraints': []},
   {'name': 'Education', 'indexes': ['name'], 'constraints': []},
   {'name': 'Preference', 'indexes': ['name'], 'constraints': []},
   {'name': '__Entity__',
    'indexes': [],
    'constraints': ["Constraint( id=4, name='constraint_907a464e', type='UNIQUENESS', schema=(:__E

In [15]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Classification {name: STRING}
Organization {name: STRING}
Education {name: STRING}
Experience {name: STRING}
SecurityScreening {name: STRING}
Accountability {name: STRING}
Role {name: STRING}
Preference {name: STRING}
JobProfile {views: INTEGER, updated_at: DATE_TIME, context: STRING, title: STRING, created_at: DATE_TIME, number: INTEGER, all_reports_to: BOOLEAN, valid_from: DATE_TIME, id: INTEGER, type: STRING, is_archived: BOOLEAN, version: INTEGER, overview: STRING, state: STRING, published_at: DATE_TIME, program_overview: STRING}
BehavioralCompetency {name: STRING}
Registration {name: STRING}
KSA {name: STRING}
WillingnessStatement {name: STRING}
RoleType {name: STRING}
Scope {name: STRING}
JobFamily {name: STRING}
Stream {name: STRING}
ReportsTo {name: STRING}
Relationship properties:

The relationships:
(:JobProfile)-[:HAS_ACCOUNTABILITY]->(:Accountability)
(:JobProfile)-[:HAS_CLASSIFICATION]->(:Classification)
(:JobProfile)-[:BELONGS_TO_ORGANIZATION]->(:Organiza

In [16]:
# Check relationship count in database
result = graph.query("""
    MATCH ()-[r]->() 
    RETURN count(r) AS relationship_count,
           collect(distinct type(r)) AS relationship_types
""")
print(f"Relationships Found: {result[0]['relationship_count']}")
print(f"Relationship Types: {result[0]['relationship_types']}")

Relationships Found: 38206
Relationship Types: ['REQUIRES_EXPERIENCE', 'REQUIRES_KSA', 'REQUIRES_SCREENING', 'HAS_ROLE', 'HAS_ROLE_TYPE', 'HAS_CLASSIFICATION', 'BELONGS_TO_ORGANIZATION', 'HAS_SCOPE', 'BELONGS_TO_JOB_FAMILY', 'BELONGS_TO_STREAM', 'REPORTS_TO', 'HAS_COMPETENCY', 'HAS_ACCOUNTABILITY', 'REQUIRES_EDUCATION', 'HAS_WILLINGNESS', 'REQUIRES_REGISTRATION', 'HAS_PREFERENCE']


## Build vector index

Vectorstore generation from existing graph

In [19]:
# Add to existing Neo4jGraph initialization
from langchain_community.vectorstores import Neo4jVector
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

# his method pulls relevant text information from the database, and calculates and stores the text embeddings back to the database.
vector_store = Neo4jVector.from_existing_graph(
    embedding=embeddings,
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    index_name="document_embeddings",
    embedding_node_property="embedding",
    node_label="Entity", # Generic label for all nodes
    text_node_properties=["name"]
)

In [20]:
index_info = vector_store.query("""
SHOW INDEXES 
WHERE name = 'document_embeddings'
""")
print(index_info)

[{'id': 5, 'name': 'document_embeddings', 'state': 'ONLINE', 'populationPercent': 100.0, 'type': 'VECTOR', 'entityType': 'NODE', 'labelsOrTypes': ['Document'], 'properties': ['embedding'], 'indexProvider': 'vector-2.0', 'owningConstraint': None, 'lastRead': neo4j.time.DateTime(2025, 2, 24, 7, 49, 10, 9000000, tzinfo=<UTC>), 'readCount': 2}]


In [21]:
from pprint import pprint


results = vector_store.similarity_search("Public Safety & Sol General (PSSG)", k=2)
for doc in results:
    print('==== DOC ====')
    pprint(doc.metadata)

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

# Initialize with updated model naming format

llm = AzureAIChatCompletionsModel(
            endpoint=os.getenv('AZURE_ENDPOINT'),
            credential=os.getenv('AZURE_API_KEY'),
            model_name="Mistral-small",
            api_version="2024-05-01-preview",
            model_kwargs={"max_tokens": 4000},
            
            temperature=0.5,
            top_p=0.4
        )
graph_transformer = LLMGraphTransformer(llm=llm)
chain = GraphCypherQAChain.from_llm(
    llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True,
    # exclude_types=['Document']
    # validate_cypher=True,  # New critical parameter
    # schema_constraints={
    #     "Jobprofile": {"identifier": "id"},  # Force 'id' usage
    #     "Document": {"identifier": "title"}
    # }
)
chain.invoke({"query": "What organizations does the 'Licensing Clerk' profile belong to? Ensure title is treated as 'id' instead of 'title'"})
# use backticks for labels containing spaces: e.g. MATCH (jt:`Job Title` ========== \n 